Initial thoughts: 
 - Many features, feature engineering will be required. Manually eliminate/combine sum to reduce dimensions of feature set, then use techniques to try and reduce further
 - Once initial features decided upon, lets start with multiple linear regression - check if we have multicollinearity

In [1]:
import numpy as np
from scipy import stats
from math import log
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
import seaborn as sns
from IPython.display import display
pd.options.display.max_columns = None

In [171]:
# Read in the train and test data
train_data = pd.read_csv("data/train.csv", index_col = 0)
sale_price = train_data[['SalePrice']]
test_data = pd.read_csv("data/test.csv", index_col = 0)
test_data['SalePrice'] = 0

In [172]:
all_data = pd.concat([train_data, test_data], axis = 0).reset_index(drop = True)

###### Target is highly skewed! Use a transformation to correct

In [ ]:
# Compare normality of target under no transformation, optimum box-cox and log
fig1, ax1 = plt.subplots(figsize=(11.7, 8.27))
prob1 = stats.probplot(sale_price['SalePrice'], dist=stats.norm, plot=ax1)
ax1.set_title('Probplot against normal distribution')

print(all(sale_price['SalePrice'] > 0))
fig2, ax2 = plt.subplots(figsize=(11.7, 8.27))
sale_price_log = stats.boxcox(sale_price, lmbda = 0)
prob2 = stats.probplot(sale_price_log.flatten(), dist=stats.norm, plot=ax2)
ax2.set_title('Probplot after log transformation')

fig3, ax3 = plt.subplots(figsize=(11.7, 8.27))
sale_price_bc, lmbda = stats.boxcox(sale_price)
prob3 = stats.probplot(sale_price_bc.flatten(), dist=stats.norm, plot=ax3)
ax3.set_title('Probplot after Box-Cox transformation')

plt.show()
# Optimal Lambda is -0.07692396

In [ ]:
# Plot histograms of the lob and box-cos transformed target data
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(11.7, 8.27))
ax0, ax1, = axes.flatten()
ax0.hist(sale_price_log, density = True)
ax1.hist(sale_price_bc, density = True)

###### The target data transformed by the logarithm (lambda = 0) looks pretty good and is more easily interpreted - continue with this

### Missingness

In [ ]:
# Get an overview of the feature set
all_data.describe()
# 36 numerical variables

In [135]:
all_data.columns[all_data.isnull().any()]
# 25 features with missing data

Index(['MSZoning', 'LotFrontage', 'Alley', 'Utilities', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'Electrical', 'BsmtFullBath',
       'BsmtHalfBath', 'KitchenQual', 'Functional', 'FireplaceQu',
       'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea',
       'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType'],
      dtype='object')

In [ ]:
# View the columns with missing data
all_data.loc[:, ['MSZoning', 'LotFrontage', 'Alley', 'Utilities', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'Electrical', 'BsmtFullBath',
       'BsmtHalfBath', 'KitchenQual', 'Functional', 'FireplaceQu',
       'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea',
       'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType']]

In [ ]:
# Visualize missing data combinations
fig, ax = plt.subplots(figsize=(11.7, 8.27))
sns.heatmap(train_data.loc[:, ['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence',
       'MiscFeature']].isnull(), cbar = False, ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(11.7, 8.27))
sns.heatmap(train_data.loc[:, ['LotFrontage', 'Alley', 'FireplaceQu', 'Fence']].isnull(), cbar = False, ax=ax)

(2919, 80)

RL         2265
RM          460
FV          139
RH           26
C (all)      25
Name: MSZoning, dtype: int64


Pave    2907
Grvl      12
Name: Street, dtype: int64


Grvl    120
Pave     78
Name: Alley, dtype: int64


Reg    1859
IR1     968
IR2      76
IR3      16
Name: LotShape, dtype: int64


Lvl    2622
HLS     120
Bnk     117
Low      60
Name: LandContour, dtype: int64


AllPub    2916
NoSeWa       1
Name: Utilities, dtype: int64


Inside     2133
Corner      511
CulDSac     176
FR2          85
FR3          14
Name: LotConfig, dtype: int64


Gtl    2778
Mod     125
Sev      16
Name: LandSlope, dtype: int64


NAmes      443
CollgCr    267
OldTown    239
Edwards    194
Somerst    182
NridgHt    166
Gilbert    165
Sawyer     151
NWAmes     131
SawyerW    125
Mitchel    114
BrkSide    108
Crawfor    103
IDOTRR      93
Timber      72
NoRidge     71
StoneBr     51
SWISU       48
ClearCr     44
MeadowV     37
BrDale      30
Blmngtn     28
Veenker     24
NPkVill     23
Blueste     10
Name

### Data cleaning and preliminary feature selection

In [203]:
# Replace the missing values from the basement data with random selections from the appropriate options,
# because there are very few of these situations so knn seems unnecessary
np.random.seed(0)
basement = all_data.loc[:,['BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',]]
basement[(basement.isnull().any(axis = 1)) & (basement.loc[:,'TotalBsmtSF'] != 0)] =\
basement[(basement.isnull().any(axis = 1)) & (basement.loc[:,'TotalBsmtSF'] != 0)].apply(lambda x: x.fillna(np.random.choice(x.dropna())), axis=0)

# Now replace the NA's that correspond to no garage with "None"
basement.loc[basement.isnull().any(axis = 1),['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']] = "None"
all_data.loc[:,['BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',]] = basement

##### Need to do something about index 2120 - currently totally random values picked for basement features####

In [187]:
# Remove the singular missing completely at random observation in the electrical column
all_data = all_data.drop(all_data[all_data.loc[:,'Electrical'].isnull()].index[0], axis = 0)

In [159]:
# Replace all NA values that actually mean no fireplaces with 'None'
all_data.loc[(all_data['FireplaceQu'].isnull()) & (all_data['Fireplaces'] == 0),'FireplaceQu'] = "None"

FireplaceQu  Fireplaces
0           None           0
1             TA           1
2             TA           1
3             Gd           1
4             TA           1
5           None           0
6             Gd           1
7             TA           2
8             TA           2
9             TA           2
10          None           0
11            Gd           2
12          None           0
13            Gd           1
14            Fa           1
15          None           0
16            TA           1
17          None           0
18          None           0
19          None           0
20            Gd           1
21            Gd           1
22            Gd           1
23            TA           1
24            TA           1
25            Gd           1
26          None           0
27            Gd           1
28            Gd           2
29          None           0
...          ...         ...
2889        None           0
2890        None           0
2891        None           0
2892        None           0
2893        None           0
2894          Gd           1
2895          Gd           1
2896          TA           1
2897        None           0
2898        None           0
2899        None           0
2900          Gd           2
2901        None           0
2902          Gd           2
2903          Gd           1
2904        None           0
2905        None           0
2906          TA           1
2907        None           0
2908        None           0
2909        None           0
2910        None           0
2911          TA           1
2912        None           0
2913        None           0
2914        None           0
2915        None           0
2916          TA           1
2917        None           0
2918          TA           1

[2918 rows x 2 columns]

In [169]:
# Replace all NA values that actually mean no garage with 'None'
all_data.loc[(all_data['GarageType'].isnull()) &(all_data['GarageArea'] == 0),'GarageType'] = 'None'
all_data.loc[(all_data['GarageFinish'].isnull()) &(all_data['GarageArea'] == 0),'GarageFinish'] = 'None'
all_data.loc[(all_data['GarageQual'].isnull()) &(all_data['GarageArea'] == 0),'GarageQual'] = 'None'

In [168]:
# Replace all NA values that actually mean no garage with 'None'
house_test[['PoolArea','PoolQC']].loc[(house_test.PoolArea!=0) & (house_test.PoolQC.isnull())].fillna(np.random.choice(['Ex','Gd','Fa']))

False

In [ ]:
# See if any of the features have zero variance, select only the numeric ones first
train_numeric = train_data.select_dtypes(include=[np.number])
print(train_data.apply(np.var))
train_numeric.columns[train_numeric.apply(np.var) < 1]

In [ ]:
 # Investigate those features with very low variance
train_data.loc[:,['BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'Fireplaces', 'GarageCars']].apply(pd.Series.value_counts)

In [ ]:
# Investigate the value counts of all the categorical columns
all_categorical = all_data.select_dtypes(exclude=[np.number])
all_data.shape

In [158]:
for col in all_categorical.columns:
    print(all_categorical[str(col)].value_counts())
    print('\n')

RL         2265
RM          460
FV          139
RH           26
C (all)      25
Name: MSZoning, dtype: int64


Pave    2907
Grvl      12
Name: Street, dtype: int64


Grvl    120
Pave     78
Name: Alley, dtype: int64


Reg    1859
IR1     968
IR2      76
IR3      16
Name: LotShape, dtype: int64


Lvl    2622
HLS     120
Bnk     117
Low      60
Name: LandContour, dtype: int64


AllPub    2916
NoSeWa       1
Name: Utilities, dtype: int64


Inside     2133
Corner      511
CulDSac     176
FR2          85
FR3          14
Name: LotConfig, dtype: int64


Gtl    2778
Mod     125
Sev      16
Name: LandSlope, dtype: int64


NAmes      443
CollgCr    267
OldTown    239
Edwards    194
Somerst    182
NridgHt    166
Gilbert    165
Sawyer     151
NWAmes     131
SawyerW    125
Mitchel    114
BrkSide    108
Crawfor    103
IDOTRR      93
Timber      72
NoRidge     71
StoneBr     51
SWISU       48
ClearCr     44
MeadowV     37
BrDale      30
Blmngtn     28
Veenker     24
NPkVill     23
Blueste     10
Name